In [ ]:
!pip install pandas lifelines

In [4]:
import pandas as pd
import numpy as np

file_path = 'data/rhdnase_data.xlsx'

data_df = pd.read_excel(file_path)
data_df['duration'] = data_df['stop'] - data_df['start']
print(data_df.columns)

Index(['id', 'trt', 'fev', 'fev2', 'start', 'stop', 'event', 'etype', 'enum',
       'enum1', 'emum2', 'duration'],
      dtype='object')


In [ ]:
data_df

In [9]:
from lifelines import CoxPHFitter  
  
  
# 假设你的CSV文件有以下列：'duration', 'event', 'var1', 'var2', ... 其中'duration'是时间，'event'是事件发生（1）或未发生（0）  
# 你需要根据你的数据集来指定这些列名  
duration_col = 'duration'  
event_col = 'event'  
covariates = ['trt', 'fev', 'fev2', 'duration', 'event' ]  # 替换为你的协变量列名  
  
# 使用CoxPHFitter进行拟合  
cph = CoxPHFitter()  
cph.fit(data_df[covariates], duration_col=duration_col, event_col=event_col)  
  
# 输出模型的摘要信息  
cph.print_summary()
  
# # 你也可以预测个体的风险  
# # 例如，预测一个新的观测值的风险，其中var1=1, var2=2, var3=3  
# individual_risk = cph.predict_partial_hazard(pd.DataFrame({'trt': [1], 'fev': [2], 'fev2': [3]}))  
# print(f"Predicted partial hazard: {individual_risk[0]}")  
  
# # 你还可以预测个体的生存函数或风险函数等  
# # 例如，预测生存函数  
# survival_function = cph.predict_survival_function(pd.DataFrame({'trt': [1], 'fev': [2], 'fev2': [3]}))  
# print(survival_function.iloc[:, 0])

<lifelines.CoxPHFitter: fitted with 1326 total observations, 605 right-censored observations>
             duration col = 'duration'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 1326
number of events observed = 721
   partial log-likelihood = -4820.53
         time fit was run = 2024-05-26 12:59:41 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
trt        -0.21       0.81       0.08            -0.36            -0.07                 0.70                 0.94
fev         0.05       1.05       0.04            -0.03             0.13                 0.97                 1.14
fev2       -0.06       0.94       0.04            -0.14             0.01                 0.87                 1.01

            cmp to     z      p   -log2(p)
covariate                                 
trt           0.00 -2.83 <0.005       7.73
fev           0.00  1.27   0.20       2.30
fev2          0.00 -1.60   0.11       3.20
---
Concordance = 0.58
Partial AIC = 9647.05
log-likelihood ratio test = 81.53 on 3 df
-log2(p) of ll-ratio test = 55.95